In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, Reshape
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import numpy as np
import pandas as pd
from sklearn import preprocessing 
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
!pip install numpy requests nlpaug
import pandas as pd
import numpy as np
import nlpaug.augmenter.word as nlpaw
from tqdm import tqdm

     |████████████████████████████████| 405 kB 596 kB/s eta 0:00:01


In [4]:
!pip install transformers==3.0.0

     |████████████████████████████████| 754 kB 606 kB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 13.6 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.6.1
    Uninstalling transformers-4.6.1:
      Successfully uninstalled transformers-4.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.5.0 requires transformers<4.7,>=4.1, but you have transformers 3.0.0 which is incompatible.


In [ ]:
dfaug.to_csv('df_aug.csv',index=False)
dfaug.head()

In [ ]:
dfaug.groupby('Pred').count()

In [5]:
dfaug = pd.read_csv('../input/newsdatasets/df_aug.csv')
max_len = 30

In [6]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
dfaug = dfaug.groupby('Pred').apply(lambda x: x.sample(4000)).reset_index(drop=True)
dfaug['Sent'] = dfaug['Sent'].astype(str)

In [7]:
from keras.utils import to_categorical
le = preprocessing.LabelEncoder()
dfaug = dfaug.sample(frac = 1)
X = dfaug['Sent'].tolist()
Y = np.array(dfaug.iloc[:]['Pred'])
le.fit(Y)
Y = le.transform(Y)
num_classes=  len(le.classes_)

In [8]:
le.classes_

array(['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire',
       'true'], dtype=object)

# Transfer Learning

In [9]:
from tqdm.notebook import tqdm
from transformers import BertTokenizer, TFBertModel, TFDistilBertModel,DistilBertTokenizer , RobertaTokenizer, TFRobertaModel
#dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
#robert_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#robert_model = TFRobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
def data_creation(X,Y,token):
    input_ids=[]
    attention_masks=[]
    for sent in tqdm(X):
        dbert_inps=token.encode_plus(str(sent),add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
        input_ids.append(dbert_inps['input_ids'])
        attention_masks.append(dbert_inps['attention_mask'])

    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    Y = np.array(Y)
    print(Y.shape)
    print(len(input_ids),len(attention_masks),len(Y))
    return input_ids,attention_masks,Y

In [11]:
model_use = bert_model
token_use = bert_tokenizer
input_ids,attention_masks,Y = data_creation(X,Y,token_use)

  0%|          | 0/24000 [00:00<?, ?it/s]

(24000,)
24000 24000 24000


In [12]:
from tensorflow.keras.layers import LSTM,add
from tensorflow.keras.layers import Flatten , Conv1D
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten , Conv1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,MaxPooling1D
from tensorflow.keras.utils import plot_model
def create_model(max_len,num_classes,mod):
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = mod(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(4096,activation='relu')(dbert_layer)
    dense = Reshape(( 256 , 16))(dense)
    head4 = Conv1D(36,5)(dense
                        )
    head4 = MaxPooling1D(5,padding='same')(head4)
    head4 = Flatten()(head4)
    dropout= Dropout(0.2)(head4)
    pred = Dense(1024, activation='softmax')(dropout)
    pred = Dropout(0.2)(pred)
    pred = Dense(64, activation='softmax')(pred)
    pred = Dropout(0.5)(pred)
    pred = Dense(32, activation='softmax')(pred)
    pred = Dropout(0.5)(pred)
    pred = Dense(num_classes, activation='softmax')(pred)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    return model

In [13]:
model = create_model(max_len,num_classes,model_use)
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <bound method TFBertModel.call of <transformers.models.bert.modeling_tf_bert.TFBertModel object at 0x7f8c965c0d90>>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <bound method TFBertMainLayer.call of <transformers.m

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True)

In [14]:
print('Preparing the pickle file.....')

pickle_inp_path='dbert_inp.pkl'
pickle_mask_path='dbert_mask.pkl'
pickle_Pred_path='dbert_Pred.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((Y),open(pickle_Pred_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_Pred_path)

Preparing the pickle file.....
Pickle files saved as  dbert_inp.pkl dbert_mask.pkl dbert_Pred.pkl


In [15]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
Preds=pickle.load(open(pickle_Pred_path, 'rb'))

print('Input shape {} Attention mask shape {} Input Pred shape {}'.format(input_ids.shape,attention_masks.shape,Preds.shape))

Loading the saved pickle files..
Input shape (24000, 30) Attention mask shape (24000, 30) Input Pred shape (24000,)


In [16]:
num_classes

6

In [17]:
train_inp,test_inp,train_Pred,test_Pred,train_mask,test_mask=train_test_split(input_ids,Preds,attention_masks,test_size=0.2)



log_dir='dbert_model'
model_save_path='./dbert_model.h5'

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

In [18]:
print('Train inp shape {} Test input shape {}\nTrain Pred shape {} Test Pred shape {}\nTrain attention mask shape {} Test attention mask shape {}'.format(train_inp.shape,test_inp.shape,train_Pred.shape,test_Pred.shape,train_mask.shape,test_mask.shape))


Train inp shape (19200, 30) Test input shape (4800, 30)
Train Pred shape (19200,) Test Pred shape (4800,)
Train attention mask shape (19200, 30) Test attention mask shape (4800, 30)


In [19]:
history=model.fit([train_inp,train_mask],train_Pred,batch_size=64,epochs=10,validation_data=([test_inp,test_mask],test_Pred))

Epoch 1/10
300/300 [==============================] - 117s 371ms/step - loss: 1.7933 - accuracy: 0.1684 - val_loss: 1.7924 - val_accuracy: 0.1656
Epoch 2/10
300/300 [==============================] - 112s 373ms/step - loss: 1.7931 - accuracy: 0.1684 - val_loss: 1.7923 - val_accuracy: 0.1656
Epoch 3/10
300/300 [==============================] - 111s 372ms/step - loss: 1.7929 - accuracy: 0.1673 - val_loss: 1.7923 - val_accuracy: 0.1656
Epoch 4/10
300/300 [==============================] - 111s 370ms/step - loss: 1.7927 - accuracy: 0.1716 - val_loss: 1.7922 - val_accuracy: 0.1656
Epoch 5/10
300/300 [==============================] - 111s 370ms/step - loss: 1.7924 - accuracy: 0.1718 - val_loss: 1.7922 - val_accuracy: 0.1656
Epoch 6/10
300/300 [==============================] - 111s 369ms/step - loss: 1.7928 - accuracy: 0.1677 - val_loss: 1.7921 - val_accuracy: 0.1656
Epoch 7/10
300/300 [==============================] - 111s 369ms/step - loss: 1.7923 - accuracy: 0.1653 - val_loss: 1.7921 -

## Evaluation

In [20]:
preds = model.predict([test_inp,test_mask],batch_size=16)
pred_Preds = preds.argmax(axis=1)
target_names=le.classes_
print('Classification Report')
print(classification_report(test_Pred,pred_Preds,target_names=target_names))
print('Training and saving built model.....')

Classification Report
              precision    recall  f1-score   support

 barely-true       0.00      0.00      0.00       790
       false       0.17      1.00      0.28       795
   half-true       0.00      0.00      0.00       802
 mostly-true       0.00      0.00      0.00       801
  pants-fire       0.00      0.00      0.00       800
        true       0.00      0.00      0.00       812

    accuracy                           0.17      4800
   macro avg       0.03      0.17      0.05      4800
weighted avg       0.03      0.17      0.05      4800

Training and saving built model.....


# Create Test from test data and remove non corpus words

In [21]:
col_list=['Loc','Pred','Sent','Topic','Speaker','Position','Region','Party','a','b','c','d','e','Occasion']
dfT = pd.read_csv('../input/newsdatasets/other_dataset/other_dataset/LIAR/test.tsv',header = None,sep='\t')
dfT.columns =  col_list
dfT.head()

,Loc,Pred,Sent,Topic,Speaker,Position,Region,Party,a,b,c,d,e,Occasion
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video


In [22]:
mask = (dfT['Sent'].str.len() < 150) 
dfT = dfT.loc[mask]
dfT.shape

(1057, 14)

In [23]:
Xt = dfT['Sent']
Yt = dfT['Pred']
Yt = le.transform(Yt)

In [25]:
dfT.shape

(1057, 14)

In [26]:
input_idsT,attention_masksT,YT = data_creation(Xt,Yt,token_use)

  0%|          | 0/1057 [00:00<?, ?it/s]

(1057,)
1057 1057 1057


In [27]:
preds = model.predict([input_idsT,attention_masksT],batch_size=16)
pred_Preds = preds.argmax(axis=1)
target_names=le.classes_
#$print('F1 score',f1)
print('Classification Report')
print(classification_report(Yt,pred_Preds,target_names=target_names))
print('Training and saving built model.....')

Classification Report
              precision    recall  f1-score   support

 barely-true       0.00      0.00      0.00       177
       false       0.21      1.00      0.35       221
   half-true       0.00      0.00      0.00       216
 mostly-true       0.00      0.00      0.00       192
  pants-fire       0.00      0.00      0.00        72
        true       0.00      0.00      0.00       179

    accuracy                           0.21      1057
   macro avg       0.03      0.17      0.06      1057
weighted avg       0.04      0.21      0.07      1057

Training and saving built model.....
